<a href="https://colab.research.google.com/github/AhmadJamal01/Floodead-Inside/blob/main/GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GANs

## Imports

In [11]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

## Read the Data

In [12]:
! rm -r gan_images/

In [13]:
from google.colab import drive
import zipfile

# Mount your Google Drive
drive.mount('/content/drive')

# Specify the path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/dataset.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/') 
 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Generator

In [14]:
# Define the generator architecture
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape, num_classes):
        super(Generator, self).__init__()

        self.embedding = nn.Embedding(num_classes, num_classes)
        
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, int(torch.prod(torch.tensor(img_shape)))),
            nn.Tanh()
        )

    def forward(self, z, labels):
        embedded_labels = self.embedding(labels)
        input = torch.cat((z, embedded_labels), dim=1)
        img = self.model(input)
        img = img.view(img.size(0), *img_shape)
        return img

## Discriminator

In [15]:
# Define the discriminator architecture
class Discriminator(nn.Module):
    def __init__(self, img_shape, num_classes):
        super(Discriminator, self).__init__()

        self.embedding = nn.Embedding(num_classes, num_classes)
        
        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))) + num_classes, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        img_flat = img.view(img.size(0), -1)
        embedded_labels = self.embedding(labels)
        input = torch.cat((img_flat, embedded_labels), dim=1)
        validity = self.model(input)
        return validity


## Configuration

In [16]:
# Speed ups
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
torch.backends.cudnn.benchmark = True

# Set device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set random seed for reproducibility
torch.manual_seed(42)

## Model

In [17]:
# Set parameters
latent_dim = 100 # the dimensionality of the input noise vector
num_classes = 2  # Number of classes (flooded and non-flooded)
img_size = 64
img_channels = 3
batch_size = 64
img_shape = (img_channels, img_size, img_size)
num_epochs = 100
sample_interval = 15
lr = 0.0002

# Initialize generator and discriminator
generator = Generator(latent_dim, img_shape, num_classes).to(device)
discriminator = Discriminator(img_shape, num_classes).to(device)

# Define loss function and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


## Data Loader

In [18]:
# Create directory for generated images
os.makedirs('gan_images', exist_ok=True)

# Define transformations to be applied to the images
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * img_channels, std=[0.5] * img_channels)
])

# Load the dataset
# Access the extracted files
path = '/content/dataset/' 
dataset = ImageFolder(root=path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# train, validation, test = torch.utils.data.random_split(dataset, (0.2, 0.4, 0.4))
# dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)


## Training

In [ ]:
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    for i, (imgs, labels) in enumerate(dataloader):

        # Adversarial ground truths
        valid = torch.ones(imgs.size(0), 1).to(device)
        fake = torch.zeros(imgs.size(0), 1).to(device)

        # Configure input
        real_imgs = imgs.to(device)
        labels = labels.to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim).to(device)

        # Generate a batch of images
        fake_imgs = generator(z, labels)

        # Measure discriminator's ability to classify real and fake images
        real_loss = adversarial_loss(discriminator(real_imgs, labels), valid)
        fake_loss = adversarial_loss(discriminator(fake_imgs.detach(), labels), fake)
        d_loss = (real_loss + fake_loss) / 2

        # Backward pass and optimize
        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim).to(device)

        # Generate a batch of images
        gen_imgs = generator(z, labels)

        # Measure generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs, labels), valid)

        # Backward pass and optimize
        g_loss.backward()
        optimizer_G.step()

        # Print progress
        if (i + 1) % sample_interval == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(dataloader)}], D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}")
            # Save generated images
            print("Saving...")
            save_image(gen_imgs.data, f"gan_images/{epoch+1}_{i+1}.png", nrow=5, normalize=True)


Epoch [1/100]
Epoch [1/100], Batch [15/15], D_loss: 0.4780, G_loss: 0.6801
Saving...
Epoch [2/100]
Epoch [2/100], Batch [15/15], D_loss: 0.3152, G_loss: 1.0011
Saving...
Epoch [3/100]
Epoch [3/100], Batch [15/15], D_loss: 0.3700, G_loss: 1.0689
Saving...
Epoch [4/100]
Epoch [4/100], Batch [15/15], D_loss: 0.3078, G_loss: 0.9890
Saving...
Epoch [5/100]
Epoch [5/100], Batch [15/15], D_loss: 0.3701, G_loss: 0.8197
Saving...
Epoch [6/100]
Epoch [6/100], Batch [15/15], D_loss: 0.3083, G_loss: 0.9261
Saving...
Epoch [7/100]
Epoch [7/100], Batch [15/15], D_loss: 0.1956, G_loss: 1.3877
Saving...
Epoch [8/100]
Epoch [8/100], Batch [15/15], D_loss: 0.1824, G_loss: 1.5442
Saving...
Epoch [9/100]
Epoch [9/100], Batch [15/15], D_loss: 0.0926, G_loss: 2.1130
Saving...
Epoch [10/100]
Epoch [10/100], Batch [15/15], D_loss: 0.0874, G_loss: 2.1612
Saving...
Epoch [11/100]
Epoch [11/100], Batch [15/15], D_loss: 0.1119, G_loss: 2.0738
Saving...
Epoch [12/100]
Epoch [12/100], Batch [15/15], D_loss: 0.1294,

In [ ]:
# Save the model
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

In [ ]:
# Load the model
generator = Generator(latent_dim, img_shape, num_classes).to(device)
discriminator = Discriminator(img_shape, num_classes).to(device)

generator.load_state_dict(torch.load('generator.pth'))
discriminator.load_state_dict(torch.load('discriminator.pth'))


In [ ]:
# Set the model to evaluation mode
generator.eval()
discriminator.eval()


In [ ]:
# Generate images using the trained generator:
num_images = 10  # Number of images to generate
labels = torch.tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])  # Example labels for generating specific class images
z = torch.randn(num_images, latent_dim).to(device)

with torch.no_grad():
    gen_imgs = generator(z, labels).detach().cpu()

# Save or display the generated images
for i in range(num_images):
    save_image(gen_imgs[i], f"generated_images/generated_{i}.png", normalize=True)


In [ ]:
# Display the Image
import matplotlib.pyplot as plt

# Rescaling Pixel Values: 
# Rescales the pixel values of the generated images from the range [-1, 1] to [0, 1]. 
# This is necessary because the generator outputs images with pixel values in the range [-1, 1] due to the Tanh activation function.
generated_imgs = (gen_imgs + 1) / 2.0

# Create a grid of images
rows = 5
cols = 5
fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

# Iterate over the grid and plot the generated images
for i, ax in enumerate(axes.flatten()):
    img = generated_imgs[i]
    ax.imshow(img.transpose(1, 2, 0))
    ax.axis('off')

# Show the plot
plt.tight_layout()
plt.show()